### Create structs: Density, stage, genetics, drive, node, network
    

In [1]:
abstract type LifeStage end
abstract type Genotype end
# abstract type DensityDependence end

In [2]:
# Preserve modularity with Density struct separate from Stage

struct Density # {D <: DensityDependence}
    K::Union{Nothing, Float64} # Logistic dependence
    γ::Union{Nothing, Float64} # Linear dependence
end


In [3]:

struct Stage{L <: LifeStage}
    stage::Type{L}
    t::Union{Nothing, Float64} # Total stage duration
    q::Union{Nothing, Float64} # 1/duration = total time in each substage 
    n::Union{Nothing, Int64}   # Number of substages 
    μ::Float64                 # Mortality 
    d::Union{Nothing, Float64} # Enables flexible density dependence specification for each stage 
end


In [4]:
# Drive = one genotype at a time (subsets Genetics)

struct Drive{G <: Genotype} 
    genotype::Type{G}            # single genotype 
    cube_slice::Array{Float64,2}      
    s::Float64          
    τ::Array{Float64,2} 
    ϕ::Float64            
    β::Float64            
    η::Float64     
end
    

In [5]:
# All genotype-specific aspects, including Reproduction parameters 

struct Genetics 
    
    all_genotypes::Array{Drive{<:Genotype}}
    cube::Array{Float64, 3}        # offspring likelihoods **, per genotype 
    S::Array{Float64,1}            # fractional reduction in fertility, per genotype 
    Τ::Array{Float64,3}            # offspring viability, per genotype  
    Φ::Array{Float64,1}            # male to female emergence ratio (gender), per genotype 
    Β::Array{Float64,1}            # female fecundity, per genotype  
    Η::Array{Float64,1}            # male mating fitness, per genotype 
        
        function Genetics(all_genotypes::Array{Drive{<:Genotype}}) 
            
            gN = length(all_genotypes) # number of different genes being considered 
            cube = Array{Float64, 3}(undef, gN, gN, gN)            
            S = Array{Float64,1}(undef, gN)
            Τ = Array{Float64, 3}(undef, gN, gN, gN)
            Φ = Array{Float64,1}(undef, gN)
            Β = Array{Float64,1}(undef, gN)
            Η = Array{Float64,1}(undef, gN)
        
            for (index, g) in enumerate(all_genotypes)
                cube[:,:,index] = g.cube_slice
                S[index] = g.s
                Τ[:,:,index] = g.τ
                Φ[index] = g.ϕ
                Β[index] = g.β
                Η[index] = g.η
            end
            
            new(all_genotypes, cube, S, Τ, Φ, Β, Η)
        
        end
    
end
   

# ** depth = distribution of offspring, horizontal slice = XX offspring for all parental combos
# Depth is genetically correct, horizontal slice more convenient mathematically and same result 
# NB instead of "Real" can do Vector{Float64} and Array{Float64,N}

In [6]:
# The fully mixing population in a single patch

struct Node 
    name::Symbol
    gene_data::Genetics      
    all_stages::Vector{Stage}
    location::Tuple{Float64,Float64}
    migration::Array{Float64,3} # tensor with each slice applicable to one state's movement  
end


In [7]:
# Network = collects all Nodes 

struct Network
    
    all_nodes::Vector{Node}
    all_locations::Vector{Tuple{Float64, Float64}}                      
    net_migration::Array{Float64,4}  # Need one matrix per "state" (genotype x stage pair) -> 
                                     # therefore one cube per node (migration for all states) ->
                                     # therefore must add a fourth dimension to encompass network 
    
    function Network(all_nodes::Vector{Node}, n_states::Int64)   
        
        n_nodes = length(all_nodes)
        all_locations = Vector{Tuple{Float64,Float64}}(undef, n_nodes)     
        net_migration = Array{Float64,4}(undef, n_nodes, n_states, n_nodes, n_nodes) 
        
        for (index, n) in enumerate(all_nodes) 
            all_locations[index] = n.location
            net_migration[index, :, :, :] = n.migration # returns a 3x3 tensor specific to each node 
        end
        
        new(all_nodes, all_locations, net_migration) 
        
    end
    
end

# Question: Best way to save "meta info" on eg node name?
# Question: How to change migration % at each time point? 
# Question: Use continuous callbacks to implement -> make sense if also using discrete for releases? Alternatives?

### Generate stage/drive specific structs 

In [8]:
# Stage specific 

struct Egg <: LifeStage end
struct Larvae <: LifeStage end
struct Pupae <: LifeStage end
struct Male <: LifeStage end
struct Female <: LifeStage end


In [9]:
# Genotype (drive characteristics) specific 

struct HH <: Genotype end 
struct Hh <: Genotype end 
struct HR <: Genotype end
struct hh <: Genotype end 
struct hR <: Genotype end 
struct RR <: Genotype end


### Insert data and build out groups 

In [10]:
# Life stages: total duration, substages, mortality, density 

stages = [
    Stage(Egg, 4., (1/4.), 4, 0.5, nothing),
    Stage(Larvae, 3., (1/3.), 8, 0.15, 355.0), 
    Stage(Pupae, 6., (1/6.), 6, 0.05, nothing),
    Stage(Male, nothing, nothing, nothing, 0.09, nothing),
    Stage(Female, nothing, nothing, nothing, 0.09, nothing),
    ]


5-element Array{Stage,1}:
 Stage{Egg}(Egg, 4.0, 0.25, 4, 0.5, nothing)                    
 Stage{Larvae}(Larvae, 3.0, 0.3333333333333333, 8, 0.15, 355.0) 
 Stage{Pupae}(Pupae, 6.0, 0.16666666666666666, 6, 0.05, nothing)
 Stage{Male}(Male, nothing, nothing, nothing, 0.09, nothing)    
 Stage{Female}(Female, nothing, nothing, nothing, 0.09, nothing)

In [11]:
# Cube slices 

layer1 = [1.0 1.0 0.50  0  0  0; 1.0 1.0 0.50  0  0  0; 0.5 0.5 0.25  0  0  0; 
        0.0 0.0 0.00  0  0  0; 0.0 0.0 0.00  0  0  0; 0.0 0.0 0.00  0  0  0]

layer2 = [0.0 0.0 0.00 1.0 0.50  0; 0.0 0.0 0.00 1.0 0.50  0; 0.0 0.0 0.00 0.5 0.25  0; 
        1.0 1.0 0.50 0.0 0.00  0; 0.5 0.5 0.25 0.0 0.00  0; 0.0 0.0 0.00 0.0 0.00  0]

layer3 = [0.0 0.0 0.50  0 0.50 1.0; 0.0 0.0 0.50  0 0.50 1.0; 0.5 0.5 0.50  0 0.25 0.5;
        0.0 0.0 0.00  0 0.00 0.0; 0.5 0.5 0.25  0 0.00 0.0; 1.0 1.0 0.50  0 0.00 0.0]

layer4 = [0  0  0 0.0 0.00  0; 0  0  0 0.0 0.00  0; 0  0  0 0.0 0.00  0; 
        0  0  0 1.0 0.50  0; 0  0  0 0.5 0.25  0; 0  0  0 0.0 0.00  0]

layer5 = [0  0  0 0.0 0.00  0; 0  0  0 0.0 0.00  0; 0  0 0.00 0.5 0.25 0.0; 
        0  0 0.50 0.0 0.50 1.0; 0  0 0.25 0.5 0.50 0.5; 0  0 0.00 1.0 0.50 0.0]

layer6 = [0  0  0 0.0 0.00  0; 0  0  0 0.0 0.00  0; 0  0 0.25  0 0.25 0.5; 
        0  0 0.00  0 0.00 0.0; 0  0 0.25  0 0.25 0.5; 0  0 0.50  0 0.50 1.0]

;

In [12]:
# Build drives = genotype, cube_slice, s, τ, ϕ, β, η 

drives = [
    Drive(HH, layer1, 1.0, ones(6,6), 0.5, 16.0, 1.0),
    Drive(Hh, layer2, 1.0, ones(6,6), 0.5, 16.0, 1.0), 
    Drive(HR, layer3, 1.0, ones(6,6), 0.5, 16.0, 1.0),
    Drive(hh, layer4, 1.0, ones(6,6), 0.5, 16.0, 1.0),
    Drive(hR, layer5, 1.0, ones(6,6), 0.5, 16.0, 1.0),
    Drive(RR, layer6, 1.0, ones(6,6), 0.5, 16.0, 1.0)    
]

;

In [13]:
testgenetics = Genetics(drives)
;

In [14]:
testgenetics.Β[1]

16.0

In [15]:
typeof(testgenetics)

Genetics

In [1]:
test = [1 2 3 4]

1×4 Array{Int64,2}:
 1  2  3  4

In [3]:
sum(test[1:2])

3

In [ ]:
Φ

In [16]:
using Random

In [17]:
# migration_matrix = number of states (stages x genotypes), number of nodes, number of nodes

migrate_matrix_node1 = rand((6*(4+8+6+1+6)), 2, 2) 
migrate_matrix_node2 = rand((6*(4+8+6+1+6)), 2, 2)
;

firstnode = :FirstNode
secondnode = :SecondNode

:SecondNode

In [18]:
# Build nodes = Name, gene_data, stages, location, migration matrix

Nodes = [Node(:FirstNode, testgenetics, stages, (37.87, 122.27), migrate_matrix_node1),
         Node(:SecomdNode, testgenetics, stages, (35.87, 120.27), migrate_matrix_node2)]

;

In [19]:
networktest = Network(Nodes, 150)

;

In [56]:
Nodes[1].gene_data.Φ

6-element Array{Float64,1}:
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5

In [21]:
Nodes[1].gene_data.all_genotypes.\

LoadError: syntax: incomplete: premature end of input

In [22]:
networktest.all_locations

2-element Array{Tuple{Float64,Float64},1}:
 (37.87, 122.27)
 (35.87, 120.27)

In [23]:
networktest.all_locations

2-element Array{Tuple{Float64,Float64},1}:
 (37.87, 122.27)
 (35.87, 120.27)

In [24]:
networktest.net_migration[1,:,:,:]

150×2×2 Array{Float64,3}:
[:, :, 1] =
 0.00838028  0.145618 
 0.204132    0.826394 
 0.763481    0.909471 
 0.484945    0.375307 
 0.156321    0.0392183
 0.0130681   0.321497 
 0.81195     0.249531 
 0.149414    0.193938 
 0.0133579   0.865424 
 0.279998    0.392699 
 0.360389    0.0569097
 0.0329383   0.160036 
 0.422093    0.993683 
 ⋮                    
 0.219175    0.778076 
 0.420166    0.261648 
 0.673056    0.641826 
 0.974306    0.284972 
 0.910672    0.648087 
 0.0481828   0.0535147
 0.921435    0.171782 
 0.10155     0.247762 
 0.759252    0.738587 
 0.696316    0.645946 
 0.422864    0.53803  
 0.444801    0.736826 

[:, :, 2] =
 0.919352   0.961327  
 0.974005   0.483321  
 0.0791198  0.250359  
 0.900824   0.0207954 
 0.826918   0.877952  
 0.0284573  0.605508  
 0.199479   0.586295  
 0.973647   0.959592  
 0.838611   0.473167  
 0.238336   0.00982556
 0.53331    0.289724  
 0.5957     0.95652   
 0.293204   0.00746403
 ⋮                    
 0.141522   0.97963   
 0.597

### EXAMPLES 

In [25]:
abstract type car_part end

In [26]:
struct spare_part{CP <: car_part}
    part_type::Type{CP}
    location::String
    quantity::Int64
    available::Bool
end

In [27]:
struct light_bulb <: car_part end
struct tire <: car_part end
struct gas_pump <: car_part end

In [28]:
parts = [
    spare_part(light_bulb, "front", 10, true),
    spare_part(tire, "back", 1, true),
    spare_part(gas_pump, "under hood", 0, false)
]

3-element Array{spare_part,1}:
 spare_part{light_bulb}(light_bulb, "front", 10, true) 
 spare_part{tire}(tire, "back", 1, true)               
 spare_part{gas_pump}(gas_pump, "under hood", 0, false)

In [29]:
function emergency(part::spare_part{light_bulb})
    return "Replace light bulb"
end

emergency (generic function with 1 method)

In [30]:
function emergency(part::spare_part{gas_pump})
    return "Panic"
end

emergency (generic function with 2 methods)

In [31]:
function emergency(part::spare_part{tire})
    return "stop and change the tire"
end

emergency (generic function with 3 methods)

In [32]:
function emergency(part::spare_part{CP}) where {CP <: car_part}
    return "I don't know this part"
end

emergency (generic function with 4 methods)

In [33]:
for p in parts
    println(emergency(p))
end

Replace light bulb
stop and change the tire
Panic


In [34]:
struct bonnet <: car_part end
hood = spare_part(bonnet, "fron", 1, false)

spare_part{bonnet}(bonnet, "fron", 1, false)

In [35]:
emergency(hood)

"I don't know this part"

In [36]:
function emergency(part::spare_part{bonnet})
    return "if your bonnet is on fire, run"

end

emergency (generic function with 5 methods)

In [37]:

stages = @view du[1:10]
for i in genotypes
    
    for s in stages
      stages= build_stage(u, s)
    end
    mate()
end

UndefVarError: UndefVarError: du not defined

In [38]:
abstract type LifeStageofBug end
abstract type Aquatic <: LifeStageofBug end

In [39]:
struct StageofBug{LF <: LifeStageofBug}
    stage_type::Type{LF}
    param1
    param2
end

In [40]:
struct egg <: Aquatic end
struct pupae <: Aquatic end
struct Adult <: LifeStageofBug end
struct Worm <: LifeStageofBug end

In [41]:
stages =[
    StageofBug(egg, 10, 2),
    StageofBug(pupae, 2, 4),
    StageofBug(Adult, 2, 3),
    StageofBug(Worm, 2,30)
]

4-element Array{StageofBug,1}:
 StageofBug{egg}(egg, 10, 2)   
 StageofBug{pupae}(pupae, 2, 4)
 StageofBug{Adult}(Adult, 2, 3)
 StageofBug{Worm}(Worm, 2, 30) 

In [42]:
function stage_model(stage::StageofBug{A}) where {A <: Aquatic}
    for i in 1:stage.param1
        println(stage.param2)
    end
    return
end

stage_model (generic function with 1 method)

In [43]:
function stage_model(stage::StageofBug{Adult})
    return "Here is an adult"
end

stage_model (generic function with 2 methods)

In [44]:
function stage_model(stage::StageofBug{Worm})
end

stage_model (generic function with 3 methods)

In [45]:
for s in stages
    (stage_model(s))
end

2
2
2
2
2
2
2
2
2
2
4
4


In [46]:
function ode_model(du, u, params, t)
    #slice u and du
    for p in population
        du[slice] = build_stages(u, p)
    end
end

ode_model (generic function with 1 method)

In [47]:
function build_stages(u, p)
    vector (same size as your du slice)
    
    for i in stage
        vector[slice] = stage(u[slice], i)
    end

    return vector
end

LoadError: syntax: space before "(" not allowed in "vector ("

In [48]:
function stage(u, p::T) where {T <: something}
    vector
    for n in p.number_of_substages
        vector[n] = f(n....)
    end
    
    return vector
end

LoadError: syntax: missing comma or ) in argument list

In [49]:
using Random

In [50]:
t = rand(10)

10-element Array{Float64,1}:
 0.8151836178209804  
 0.010063031371303044
 0.27753046335290654 
 0.34288870115506564 
 0.46059320482955113 
 0.003935771472909799
 0.02498929368481173 
 0.629033453982742   
 0.5447409841154298  
 0.5562464970876599  

In [51]:
t[3:5]

3-element Array{Float64,1}:
 0.27753046335290654
 0.34288870115506564
 0.46059320482955113